In [ ]:
#This notebook is a test for the python version. This notebook should be deleted once the python version is fully functional.

In [30]:
import sys
import os

# Dynamically add 'src' to sys.path
src_path = os.path.abspath(os.path.join(os.getcwd(), "../../../"))
sys.path.append(src_path)
from utilities.data_utils import write_to_mongodb

from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALSModel
from pyspark.sql.functions import explode, col


#MongoDB configuration

MONGO_URI = "mongodb://localhost:27017"  
DATABASE_NAME = "mind_news"
COLLECTION_NAME = "als_recommendations" 

In [31]:
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("ALSModelLoad") \
    .config("spark.master", "local[*]") \
    .config("spark.executor.memory", "2g") \
    .config("spark.mongodb.input.uri", MONGO_URI) \
    .config("spark.mongodb.output.uri", MONGO_URI) \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:10.1.1") \
    .getOrCreate()
    

24/12/27 19:18:40 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
current_working_directory = os.getcwd()
# Resolve the correct path to the model
base_directory = os.path.abspath(os.path.join(current_working_directory, "../../../"))
model_path = os.path.join(base_directory, "saved_models", "als_model")

# Load the ALS model
als_model = ALSModel.load(model_path)

In [33]:
# Perform recommendations
recommendations = als_model.recommendForAllUsers(numItems=10)
recommendations.show()


+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    12|[{26706, 0.066153...|
|    22|[{47061, 0.032549...|
|    31|[{35729, 0.800006...|
|    53|[{35729, 0.898198...|
|    65|[{42977, 0.724066...|
|    81|[{63970, 0.824122...|
|    91|[{49685, 0.855558...|
|   103|[{26262, 0.164236...|
|   108|[{26262, 0.210001...|
|   122|[{61233, 0.556312...|
|   126|[{13930, 0.475826...|
|   128|[{55689, 0.943486...|
|   137|[{42457, 0.405569...|
|   155|[{23446, 0.772672...|
|   159|[{287, 0.01659388...|
|   183|[{61233, 0.235151...|
|   190|[{6837, 0.3602558...|
|   192|[{23446, 0.839292...|
|   193|[{58363, 0.876614...|
|   211|[{18708, 0.092235...|
+------+--------------------+
only showing top 20 rows



In [34]:
# Explode the 'recommendations' column
exploded_recommendations = recommendations.select(
    col("userId"),
    explode(col("recommendations")).alias("recommendation")
)

# Extract 'itemId' and 'rating' from the 'recommendation' column
final_recommendations = exploded_recommendations.select(
    col("userId"),
    col("recommendation.newsId").alias("recommendation"),
    col("recommendation.rating").alias("rating")
)

# Show the transformed DataFrame
final_recommendations.show(truncate=False)

+------+--------------+-----------+
|userId|recommendation|rating     |
+------+--------------+-----------+
|12    |26706         |0.066153154|
|12    |39115         |0.059315592|
|12    |61022         |0.0582445  |
|12    |5442          |0.05216191 |
|12    |24272         |0.04952133 |
|12    |59852         |0.048400912|
|12    |33981         |0.03820189 |
|12    |27845         |0.035859898|
|12    |53017         |0.03527524 |
|12    |20495         |0.033849597|
|22    |47061         |0.032549247|
|22    |45523         |0.02985198 |
|22    |19592         |0.028439948|
|22    |6477          |0.02148725 |
|22    |287           |0.021007814|
|22    |18870         |0.018324668|
|22    |24176         |0.017829577|
|22    |14029         |0.017538551|
|22    |47098         |0.017435664|
|22    |43102         |0.017006306|
+------+--------------+-----------+
only showing top 20 rows



In [35]:
# Upload recommendations to MongoDB
write_to_mongodb(final_recommendations, MONGO_URI, DATABASE_NAME, COLLECTION_NAME)

print("Recommendations uploaded to MongoDB successfully!")

Py4JJavaError: An error occurred while calling o148.save.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: mongodb. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:725)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:873)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:260)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:251)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.in